In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"praneethratna","key":"4997eb4fb4e67eb4ec38a0e771ad035f"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 66% 17.0M/25.7M [00:00<00:00, 173MB/s]
100% 25.7M/25.7M [00:00<00:00, 165MB/s]


In [ ]:
!unzip imdb-dataset-of-50k-movie-reviews.zip

Archive:  imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df = pd.read_csv('IMDB Dataset.csv')

In [ ]:
df['sentiment'] = df['sentiment'].replace({'positive':1,'negative':0})

In [ ]:
train = df[:25000]
test = df[25000:]

In [ ]:
stemmer = PorterStemmer()
cachedstopwords = set(stopwords.words('english'))
def clean_reviews(reviews):
  cleaned_reviews = list()
  for review in reviews:
    cleaned = re.sub(r'\'s|\'|<br /><br />|[()"]|[^\w\s]','',review)
    final = cleaned.split()
    final = [word.lower() for word in final]
    final = [word for word in final if word not in cachedstopwords]
    final = [word for word in final if word.isalpha()]
    final = [stemmer.stem(word) for word in final]
    sentence = ' '.join(final)
    cleaned_reviews.append(sentence)
  return cleaned_reviews  

In [ ]:
pos_reviews = list(train[train['sentiment']==1]['review'])
neg_reviews = list(train[train['sentiment']==0]['review'])
cleaned_reviews = clean_reviews(list(train['review']))

In [ ]:
vocab = list()
for review in cleaned_reviews:
  for word in review.split():
    vocab.append(word)

In [ ]:
def word_count(sentences,sentiment,dictionary):
  for sentence , sentiment in zip(sentences,sentiment):
    for w in sentence.split():
      if (w,sentiment) in dictionary.keys():
        dictionary[(w,sentiment)] = dictionary.get((w,sentiment)) + 1
      else:
        dictionary[(w,sentiment)] = 1
  return dictionary       

In [ ]:
word_count = word_count(cleaned_reviews,list(train['sentiment']),dict())

In [ ]:
def get_count(pair,word_counts):
  c = 0
  if pair in word_counts.keys():
    c = word_counts[pair]
  return c  

In [ ]:
from math import log
V = len(set(vocab))
Dpos = len(pos_reviews)
Dneg = len(neg_reviews)
def prob_calculate(vocab_size,word_counts,vocab):
  Npos = 0
  Nneg = 0
  probs = dict()
  loglikelihood = dict()
  for word , key in word_counts.keys():
    if key == 1:
      Npos += word_counts[(word,key)]
    else:
      Nneg += word_counts[(word,key)]
  for word in set(vocab):
      probs[(word,1)] = (get_count((word,1),word_counts) + 1)/(Npos + vocab_size)
      probs[(word,0)] = (get_count((word,0),word_counts) + 1)/(Nneg + vocab_size)
      loglikelihood[word] = log((probs[(word,1)]/probs[(word,0)]))            
  return probs , loglikelihood, Npos, Nneg      

In [ ]:
probs , loglikelihood , npos, nneg = prob_calculate(V,word_count,vocab)

In [ ]:
def classifier(reviews):
  preprocessed = clean_reviews(reviews)
  logprior = log(Dpos/Dneg)
  predictions = list()
  for review in preprocessed:
    prediction = 0
    words = review.split()
    for word in words:
      if word in loglikelihood.keys():
        prediction += loglikelihood[word]
    if logprior + prediction > 0:
      predictions.append(1)
    else:
      predictions.append(0)       
  return predictions

In [116]:
predictions = classifier(list(test['review']))
y_true = test['sentiment']
correct = np.sum(np.array(y_true) == np.array(predictions))
accuracy = (correct * 100)/len(test)
print("The accuracy is {}%".format(accuracy))

The accuracy is 84.76%


In [117]:
review = ["it`s is a veryy bad movie as it did not have a pretty good storyline and it was just dragged till the end"]
prediction = classifier(review)
print(prediction[0])

0
